# Stock price prediction using BERT and GAN

based on the article: 
Priyank Sonkiya & Vikas Bajpai & Anukriti Bansal, 2021. "Stock price prediction using BERT and GAN," Papers 2107.09055, arXiv.org.


## Libraries

In [ ]:
!pip install yfinance

# Import

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import urllib.request
import zipfile
from IPython import display

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)

# Apple Stock Price (yfinance)

In [ ]:
import yfinance as yf

symbol = 'AAPL'
df = yf.download(symbol, start='2010-01-01', end='2021-12-31')

# Technical Indicators


In [ ]:
plt.rcParams["figure.figsize"]=20,10

plt.title(symbol)
plt.xlabel('Date')
plt.ylabel('Closing Price (USD)')
plt.plot(df.index, df['Close'], label='Closing Prices')
plt.show()

**Bollinger Bands**


In [ ]:
ticker = df[('2021-01-01' <= df.index) & (df.index <= '2021-12-31')]

# Typical Price
tp = (ticker['Close'] + ticker['Low'] + ticker['High'])/3

# Simple Moving Average
sma7 = tp.rolling(7).mean()
sma21 = tp.rolling(21).mean()

sma20 = tp.rolling(20).mean()
std20 = tp.rolling(20).std(ddof=0)

bollinger_up = sma20 + 2 * std20
bollinger_down = sma20 - 2 * std20

plt.plot(ticker.index, ticker['Close'], label='Closing Price', color='blue')
plt.plot(sma7, label='7 Day SMA', color='green', linestyle='--')
plt.plot(sma21, label='21 Day SMA', color='red', linestyle='--')
plt.plot(bollinger_up, label='Bollinger Up', color='cyan')
plt.plot(bollinger_down, label='Bollinger Down', color='cyan')
plt.fill_between(ticker.index, bollinger_up, bollinger_down, color='blue', alpha=0.1)

plt.title(symbol + ' - 2021')
plt.xlabel('Date')
plt.ylabel('Closing Price (USD)')
plt.legend()
plt.show()

**Moving Average Convergence Divergence** 

In [ ]:
exp1 = ticker['Close'].ewm(span=12, adjust=False).mean()
exp2 = ticker['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
exp3 = macd.ewm(span=9, adjust=False).mean()

fig, ax1 = plt.subplots()
ax1.plot(ticker['Close'], label='Closing Price', color='blue')
ax2 = ax1.twinx()
ax2.plot(exp3, label='Signal Line', color='green')
ax2.plot(macd, label=symbol + ' MACD', color='red')

ax1.set_ylabel('Closing Price (USD)')
ax2.set_ylabel('MACD')
plt.title(symbol + ' - 2021')
plt.xlabel('Date')
lines = ax1.get_lines() + ax2.get_lines()
plt.legend(lines, [l.get_label() for l in lines], loc='upper left')
plt.show()

**Relative Strength Index**

In [ ]:
delta = ticker['Close'].diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up / ema_down
rsi = 100 - (100 / (1 + rs))

# Skip first 14 days to have real values
# ticker_skip14 = ticker.iloc[14:]
fig, ax1 = plt.subplots()
line1, = ax1.plot(ticker['Close'], label='Closing Price', color='blue')
ax2 = ax1.twinx()
line2, = ax2.plot(rsi, label='RSI', color='purple')
ax2.set_ylim(0,100)
ax2.axhline(30, color='green', linestyle=':')
ax2.axhline(50, color='orange', linestyle=':')
ax2.axhline(70, color='red', linestyle=':')
ax2.axhspan(30, 70, facecolor='blue', alpha=0.1)

ax1.set_ylabel('Closing Price (USD)')
ax2.set_ylabel('RSI')
plt.title(symbol + ' - 2021')
plt.xlabel('Date')
plt.legend([line1, line2], [line1.get_label(), line2.get_label()], loc='upper left')
plt.show()